In [2]:
#boot up libraries
import pandas as pd
from sklearn import preprocessing

In [3]:
#load up full csv
df = pd.read_csv("cleaned_data/complete_merged_table.csv")

In [4]:
df.head()

,County,Year,Total_Population,Median_Income,White_Perc,Poverty_Perc,Uninsured_Perc,Asthma_Incidents,Asthma_Rate,Acres_Burned,Duration,Wildfire_Count,AQ_Min,AQ_Max,AQ_Mean,AQ_Median
0,Alameda,2011,1656754.0,43583.0,0.314137,0.097529,0.03743,6706,56.6,0.0,0,0.0,1.0,135.0,36.061303,33.0
1,Alameda,2012,1656754.0,43583.0,0.314137,0.097529,0.03743,6965,58.4,0.0,0,0.0,0.0,96.0,31.979620,30.0
2,Alameda,2013,1656754.0,43583.0,0.314137,0.097529,0.03743,6705,55.0,478.0,0,4.0,0.0,119.0,40.067273,38.0
3,Alameda,2014,1656754.0,43583.0,0.314137,0.097529,0.03743,6332,51.6,0.0,0,0.0,0.0,119.0,33.979899,31.0
4,Alameda,2015,1656754.0,43583.0,0.314137,0.097529,0.03743,6513,53.1,3340.0,5,5.0,0.0,124.0,36.424754,33.0


In [13]:
#had to remove NA values and county in order to perform normalization metho
df2 = df[['Median_Income', 'AQ_Mean']].dropna()

In [14]:
_norm = preprocessing.normalize(df2, axis=0)

In [17]:
name = ['Median_Income', 'AQ_Mean']
df3 = pd.DataFrame(_norm, columns = name)
df3

,Median_Income,AQ_Mean
0,0.087752,0.060655
1,0.087752,0.053789
2,0.087752,0.067393
3,0.087752,0.057154
4,0.087752,0.061266
...,...,...
250,0.061070,0.050488
251,0.061070,0.044068
252,0.061070,0.054496
253,0.061070,0.067108
